# Descargar Modelo Digital de Elevación 

In [15]:
# Importar GEE y Iniciar sesion
import ee
import geemap
import os
import geopandas as gpd
import glob

In [16]:
ee.Initialize()

In [17]:
Map = geemap.Map(basemap='Esri.WorldImagery')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Importar shapefile

In [18]:
os.getcwd()

'C:\\Users\\GEOMATICA\\Curso_Geomatica\\06_BGR_Geomatica\\Datos\\Shp'

In [19]:
ruta_archivos = r"C:\Users\GEOMATICA\Curso_Geomatica\06_BGR_Geomatica\Datos\Shp"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'C:\\Users\\GEOMATICA\\Curso_Geomatica\\06_BGR_Geomatica\\Datos\\Shp'

In [20]:
lista = glob.glob("*.shp")
lista

['Zona_Parcela.shp']

In [21]:
# cargar un shp
zona_gdp = gpd.read_file(lista[0])
zona_gdp

,Id,geometry
0,0,"POLYGON ((522298.171 9028900.509, 520737.679 9..."


## Convertir en Geográfico WGS84

In [26]:
zona_gdp.crs.name

'WGS 84 / UTM zone 18S'

In [27]:
# Hagamos una copia de seguridad de nuestros datos
Zona_geo = zona_gdp.copy()

# Reproject the data
Zona_wg84 = Zona_geo.to_crs(epsg=4326)

In [28]:
Zona_wg84.crs.name

'WGS 84'

## Convertir de Geopandas a GEE

In [29]:
# Crear un featureCollecion mediante GeoJSON
ee_zona_gdp = ee.FeatureCollection(Zona_wg84.__geo_interface__)

In [30]:
geometria_zona = ee_zona_gdp.geometry()

In [31]:
type(geometria_zona)

ee.geometry.Geometry

In [34]:
# Visualizar mapa
Map.centerObject(geometria_zona,12)
Map.addLayer(ee_zona_gdp, {"color" : "00FF11"}, "zona_gdp")
Map

Map(bottom=275263.0, center=[-8.767445226856395, -74.77254035437262], controls=(WidgetControl(options=['positi…

In [43]:
Bounds = geometria_zona.bounds()

In [48]:
Buffer = Bounds.buffer(300)

In [49]:
Map.addLayer(Buffer, {"color" : "blue"}, "Buffer bouns")
Map

Map(bottom=1100165.0, center=[-8.769894252609031, -74.76445198059083], controls=(WidgetControl(options=['posit…

## Obtencion de MDE

In [51]:
SRTM_3 = ee.Image("USGS/SRTMGL1_003")\
            .select('elevation')

In [52]:
# Recortar RS y TB
MDE_Buffer = SRTM_3.clip(Buffer)

In [53]:
# Reproyectar segun zona de estudio
MDE_Buffer_utm = MDE_Buffer.reproject(crs="EPSG:32718", scale=30)

In [54]:
# Consultar Proyeccion
print("Proyeccion MDE:",MDE_Buffer_utm .projection().getInfo())

Proyeccion MDE: {'type': 'Projection', 'crs': 'EPSG:32718', 'transform': [30, 0, 0, 0, -30, 0]}


In [55]:
# Visualizacion TB y RS
Map.addLayer(MDE_Buffer_utm, {},"MDE Utm")
Map

Map(bottom=1100165.0, center=[-8.769894252609031, -74.76436614990236], controls=(WidgetControl(options=['posit…

## Exportar Imagen calibrado

In [56]:
os.getcwd()

'C:\\Users\\GEOMATICA\\Curso_Geomatica\\06_BGR_Geomatica\\Datos\\Shp'

In [57]:
ruta_archivos = r"D:\Descarga"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'D:\\Descarga'

In [58]:
# Exportar imagen
geemap.ee_export_image(MDE_Buffer_utm, 
                       filename="MDE_SRTM.tif", 
                       scale=30, 
                       region=geometria_zona, 
                       file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to D:\Descarga
